# TCGA-BRCA - Load the clinical data file and keep only the necessary columns

In [1]:
import pandas as pd
import os

dataPath = '../Data'
dataFile = os.path.join(dataPath, '8162d394-8b64-4da2-9f5b-d164c54b9608', 'nationwidechildrens.org_clinical_patient_brca.txt')

df = pd.read_csv(filepath_or_buffer=dataFile, sep='\t', skiprows=[1,2])
# Note: The clinical file has a three-line header; the second and third lines are not needed for our pusposes
df[['bcr_patient_uuid', 'er_status_by_ihc', 'pr_status_by_ihc', 'her2_status_by_ihc']] # Expected output: 1097 rows

,bcr_patient_uuid,er_status_by_ihc,pr_status_by_ihc,her2_status_by_ihc
0,6E7D5EC6-A469-467C-B748-237353C23416,Positive,Positive,Negative
1,55262FCB-1B01-4480-B322-36570430C917,Positive,Positive,Positive
2,427D0648-3F77-4FFC-B52C-89855426D647,Positive,Positive,Indeterminate
3,C31900A4-5DCD-4022-97AC-638E86E889E4,Positive,Positive,Positive
4,6623FC5E-00BE-4476-967A-CBD55F676EA6,Positive,Positive,Equivocal
...,...,...,...,...
1092,5CD79093-1571-4F71-8136-0D84CCABDCAC,Positive,Positive,Negative
1093,F89588E9-CA73-4465-A7FB-7246EDB45E3A,Positive,Positive,Negative
1094,CA20249F-B7EA-4FD9-9ECB-34F74755AE35,Positive,Positive,Negative
1095,23F438BD-1DBB-4D46-972F-1E8E74DDBD37,Positive,Positive,Negative


## Determine Triple Negative status and drop indeterminable cases

In [2]:
triple_negative = (df[['er_status_by_ihc', 'pr_status_by_ihc', 'her2_status_by_ihc']] == 'Negative').all(axis=1)
any_positive = (df[['er_status_by_ihc', 'pr_status_by_ihc', 'her2_status_by_ihc']] == 'Positive').any(axis=1)
df['tnbc'] = pd.NA
df.loc[triple_negative, 'tnbc'] = True
df.loc[any_positive, 'tnbc'] = False
# df[df.isna().any(axis=1)] # Shows indeterminable cases: 118 rows x 5 columns
df.dropna(inplace=True) # Drop indeterminable cases
df['tnbc'].value_counts() # Expected tnbc values: False 863, True 116

tnbc
False    863
True     116
Name: count, dtype: int64

## Link all cases to the corresponding RNA sequencing data file using the metadata

### 2 cases have no RNA files; 1 TNBC case, 1 nTNBC case

In [3]:
import json

with open(os.path.join(dataPath, 'metadata.cart.2025-04-01.json'), 'r') as f:
    data = json.load(f)

rna_files = {
    entity['case_id'].upper(): os.path.join(file['file_id'], file['file_name'])
    for file in data
    if 'experimental_strategy' in file and file['experimental_strategy'] == 'RNA-Seq' and 'associated_entities' in file and len(file['associated_entities']) > 0
    for entity in file['associated_entities']
} # case_id in uppercase equals df['bcr_patient_uuid']; file is found in folder with its 'file_id', under its 'file_name'
rna_files

df['file'] = df['bcr_patient_uuid'].str.upper().map(rna_files)
df['exists'] = df['file'].apply(lambda file: not pd.isna(file) and os.path.exists(os.path.join(dataPath, file)))
# df[df['exists'] == False] # Shows cases with missing RNA files: 2 rows
df = df[df['exists'] == True]
print(df['tnbc'].value_counts())
df[['bcr_patient_uuid', 'tnbc', 'file']] # Expected output: 977 rows

tnbc
False    862
True     115
Name: count, dtype: int64


,bcr_patient_uuid,tnbc,file
0,6E7D5EC6-A469-467C-B748-237353C23416,False,84225715-14a6-423c-a6d6-15558e151f56\253aa5dc-...
1,55262FCB-1B01-4480-B322-36570430C917,False,927604f9-a38e-4c3f-b50f-3e0db4daf5ec\1be6a56c-...
2,427D0648-3F77-4FFC-B52C-89855426D647,False,931442ba-af81-4b68-beca-7285fc44b1df\f2dda955-...
3,C31900A4-5DCD-4022-97AC-638E86E889E4,False,7b4d770a-2b8c-4ca5-bf51-c4745c5de39a\ae8996bd-...
4,6623FC5E-00BE-4476-967A-CBD55F676EA6,False,307261f2-f88f-4658-b6d1-98ef946148e2\75d91076-...
...,...,...,...
1092,5CD79093-1571-4F71-8136-0D84CCABDCAC,False,0a7dc8b9-4196-41d3-ada1-a50cb36bfd2b\3c9c665e-...
1093,F89588E9-CA73-4465-A7FB-7246EDB45E3A,False,e0cb738d-854a-4033-b370-79fa28d7cef8\69118aea-...
1094,CA20249F-B7EA-4FD9-9ECB-34F74755AE35,False,e25bb12b-f88b-41c5-951e-a36d4b94008f\73e13f2d-...
1095,23F438BD-1DBB-4D46-972F-1E8E74DDBD37,False,ee447251-5c90-426f-a0c7-bd2041189761\06a03e45-...


# Write TCGA-BRCA data

In [4]:
df.to_csv(os.path.join(dataPath, 'clinical.csv'), index=False)

# Load Gene Expression Omnibus data

## GeoData - File MetaData

In [17]:
geoData = {}

geoData['GSE65216'] = {'dataSetName': 'GSE65216', 'dataSetFile':'GSE65216-GPL570_series_matrix.txt'   , 'dataSetSkipRows': 67 , 'dataSetSkipFooter': 1, 'platformName': 'GPL570'  , 'platformFile': 'GPL570-55999.txt'  , 'platformSkipRows': 16, 'platformSkipFooter': 0, 'processing':'Data were analyzed using standard AffyCDF or Brainarray HGU133Plus2_Hs_ENTREZG version 13 custom chipset definition file. Data were normalized using GC-RMA. Batch and hybridization effects were corrected using a linear model in which they were treated as fixed effects. Samples with technical replicates were subsequently averaged.'}
geoData['GSE76250'] = {'dataSetName': 'GSE76250', 'dataSetFile':'GSE76250_series_matrix.txt'          , 'dataSetSkipRows': 74 , 'dataSetSkipFooter': 1, 'platformName': 'GPL17586', 'platformFile': 'GPL17586-45144.txt', 'platformSkipRows': 15, 'platformSkipFooter': 1, 'processing':'The data were analyzed with Robust Multichip Analysis (RMA) algorithm using Affymetrix default analysis settings and global scaling as normalization method (ExpressionConsole 1.3.1) probe group file: HTA-2_0.r1.pgf meta-probeset file: HTA-2_0.r1.Psrs.mps'}

## GeoData - Identify TNBC / nTNBC

In [18]:
# TNBC: 55
geoData['GSE65216']['nTNBC'] = ['GSM1588972','GSM1588973','GSM1588981','GSM1588989','GSM1588990',
                                'GSM1588991','GSM1588992','GSM1588997','GSM1588998','GSM1588999',
                                'GSM1589000','GSM1589001','GSM1589002','GSM1589006','GSM1589007',
                                'GSM1589008','GSM1589013','GSM1589018','GSM1589019','GSM1589020',
                                'GSM1589021','GSM1589022','GSM1589023','GSM1589025','GSM1589026',
                                'GSM1589027','GSM1589030','GSM1589033','GSM1589034','GSM1589044',
                                'GSM1589046','GSM1589047','GSM1589048','GSM1589049','GSM1589050',
                                'GSM1589051','GSM1589052','GSM1589053','GSM1589057','GSM1589064',
                                'GSM1589065','GSM1589066','GSM1589067','GSM1589068','GSM1589069',
                                'GSM1589070','GSM1589071','GSM1589072','GSM1589073','GSM1589074',
                                'GSM1589075','GSM1589076','GSM1589077','GSM1589078','GSM1589079',
                                'GSM1589080','GSM1589081','GSM1589082','GSM1589083','GSM1589084',
                                'GSM1589085','GSM1589086','GSM1589087','GSM1589088','GSM1589089',
                                'GSM1589090','GSM1589091','GSM1589092','GSM1589093','GSM1589094',
                                'GSM1589095','GSM1589097','GSM1589098','GSM1589099','GSM1589101',
                                'GSM1589103','GSM1589105','GSM1589107','GSM1589109','GSM1589110',
                                'GSM1589111','GSM1589112','GSM1589113','GSM1589114','GSM1589115',
                                'GSM1589116','GSM1589117','GSM1589118','GSM1589119','GSM1589120',
                                'GSM1589121','GSM1589122','GSM1589123','GSM1589124','GSM1589125',
                                'GSM1589126','GSM1589127','GSM1589128','GSM1589129','GSM1589131',
                                'GSM1589133','GSM1589134','GSM1589137','GSM1589138','GSM1589140',
                                'GSM1589141','GSM1589143','GSM1589146','GSM1589147','GSM1589149',
                                'GSM1589152','GSM1589153' ]


geoData['GSE65216']['TNBC']  = ['GSM1589058','GSM1589059','GSM1589060','GSM1589061','GSM1589062',
                                'GSM1589063','GSM1589054','GSM1589055','GSM1589056','GSM1589045',
                                'GSM1589035','GSM1589036','GSM1589037','GSM1589038','GSM1589039',
                                'GSM1589040','GSM1589041','GSM1589042','GSM1589043','GSM1589031',
                                'GSM1589032','GSM1589028','GSM1589029','GSM1589024','GSM1589014',
                                'GSM1589015','GSM1589016','GSM1589017','GSM1589009','GSM1589010',
                                'GSM1589011','GSM1589012','GSM1589003','GSM1589004','GSM1589005',
                                'GSM1588993','GSM1588994','GSM1588995','GSM1588996','GSM1588982',
                                'GSM1588983','GSM1588984','GSM1588985','GSM1588986','GSM1588987',
                                'GSM1588988','GSM1588974','GSM1588975','GSM1588976','GSM1588977',
                                'GSM1588978','GSM1588979','GSM1588980','GSM1588970','GSM1588971']


In [19]:
# TNBC: 165, nTNBC: 33
geoData['GSE76250']['nTNBC'] = []

geoData['GSE76250']['TNBC']  = ['GSM1977646','GSM1977647','GSM1977648','GSM1977649','GSM1977650',
                                'GSM1977651','GSM1977652','GSM1977653','GSM1977654','GSM1977655',
                                'GSM1977656','GSM1977657','GSM1977658','GSM1977659','GSM1977660',
                                'GSM1977661','GSM1977662','GSM1977663','GSM1977664','GSM1977665',
                                'GSM1977666','GSM1977667','GSM1977668','GSM1977669','GSM1977670',
                                'GSM1977671','GSM1977672','GSM1977673','GSM1977674','GSM1977675',
                                'GSM1977676','GSM1977677','GSM1977678','GSM1977679','GSM1977680',
                                'GSM1977681','GSM1977682','GSM1977683','GSM1977684','GSM1977685',
                                'GSM1977686','GSM1977687','GSM1977688','GSM1977689','GSM1977690',
                                'GSM1977691','GSM1977692','GSM1977693','GSM1977694','GSM1977695',
                                'GSM1977696','GSM1977697','GSM1977698','GSM1977699','GSM1977700',
                                'GSM1977701','GSM1977702','GSM1977703','GSM1977704','GSM1977705',
                                'GSM1977706','GSM1977707','GSM1977708','GSM1977709','GSM1977710',
                                'GSM1977711','GSM1977712','GSM1977713','GSM1977714','GSM1977715',
                                'GSM1977716','GSM1977717','GSM1977718','GSM1977719','GSM1977720',
                                'GSM1977721','GSM1977722','GSM1977723','GSM1977724','GSM1977725',
                                'GSM1977726','GSM1977727','GSM1977728','GSM1977729','GSM1977730',
                                'GSM1977731','GSM1977732','GSM1977733','GSM1977734','GSM1977735',
                                'GSM1977736','GSM1977737','GSM1977738','GSM1977739','GSM1977740',
                                'GSM1977741','GSM1977742','GSM1977743','GSM1977744','GSM1977745',
                                'GSM1977746','GSM1977747','GSM1977748','GSM1977749','GSM1977750',
                                'GSM1977751','GSM1977752','GSM1977753','GSM1977754','GSM1977755',
                                'GSM1977756','GSM1977757','GSM1977758','GSM1977759','GSM1977760',
                                'GSM1977761','GSM1977762','GSM1977763','GSM1977764','GSM1977765',
                                'GSM1977766','GSM1977767','GSM1977768','GSM1977769','GSM1977770',
                                'GSM1977771','GSM1977772','GSM1977773','GSM1977774','GSM1977775',
                                'GSM1977776','GSM1977777','GSM1977778','GSM1977779','GSM1977780',
                                'GSM1977781','GSM1977782','GSM1977783','GSM1977784','GSM1977785',
                                'GSM1977786','GSM1977787','GSM1977788','GSM1977789','GSM1977790',
                                'GSM1977791','GSM1977792','GSM1977793','GSM1977794','GSM1977795',
                                'GSM1977796','GSM1977797','GSM1977798','GSM1977799','GSM1977800',
                                'GSM1977801','GSM1977802','GSM1977803','GSM1977804','GSM1977805',
                                'GSM1977806','GSM1977807','GSM1977808','GSM1977809','GSM1977810']


# Gather Platform data

In [20]:
import pandas as pd
import os

total = len(geoData.keys())
counter = 1
print(f"***- Reading data - Platform - Start")

for k in geoData.keys():
    dataPath = '..\\Data'
    dataFile = os.path.join(dataPath, 'geoData', geoData[k]['platformName'], geoData[k]['platformFile'])
    # engine explicit for footer
    geoData[k]['platformData'] = pd.read_csv(filepath_or_buffer=dataFile, sep='\t', skiprows=geoData[k]['platformSkipRows'], skipfooter=geoData[k]['platformSkipFooter'], engine='python')
    print(f"{counter}/{total} - Dataset {k} - Platform {geoData[k]['platformName']} data read")
    counter+=1

print(f"*** - Reading data - Platform - End")

***- Reading data - Platform - Start
1/2 - Dataset GSE65216 - Platform GPL570 data read
2/2 - Dataset GSE76250 - Platform GPL17586 data read
*** - Reading data - Platform - End


# Gather DataSet data

In [21]:
total = len(geoData.keys())
counter = 1
print(f"***- Reading data - DataSet - Start")

for k in geoData.keys():
    dataPath = '..\\Data'
    dataFile = os.path.join(dataPath, 'geoData', geoData[k]['dataSetName'], geoData[k]['dataSetFile'])
    # engine explicit for footer
    geoData[k]['dataSetData'] = pd.read_csv(filepath_or_buffer=dataFile, sep='\t', skiprows=geoData[k]['dataSetSkipRows'], skipfooter=geoData[k]['dataSetSkipFooter'], engine='python')
    print(f"{counter}/{total} - Dataset {k} - DataSet data read")
    counter+=1

print(f"***- Reading data - DataSet - End")

***- Reading data - DataSet - Start
1/2 - Dataset GSE65216 - DataSet data read
2/2 - Dataset GSE76250 - DataSet data read
***- Reading data - DataSet - End


# Join Platform data and DataSet data

### Rename DataSet data - 'ID_REF' column to 'ID'
### Rename Platform data - 'Gene Symbol' / 'gene_assignment' /'GENE_NAME' to 'GENE_SYMBOL'

In [22]:
# Rename column
geoData['GSE65216']['dataSetData'].rename(columns={"ID_REF":"ID"}, inplace=True)
geoData['GSE65216']['platformData'].rename(columns={"Gene Symbol":"GENE_SYMBOL"}, inplace=True)

# Rename column
geoData['GSE76250']['dataSetData'].rename(columns={"ID_REF":"ID"}, inplace=True)
geoData['GSE76250']['platformData'].rename(columns={"gene_assignment":"GENE_SYMBOL"}, inplace=True)

print(f"***- Rename ID & GENE_SYMBOL - Done")

***- Rename ID & GENE_SYMBOL - Done


# Make GENE_SYMBOL uniform

### Make GENE_SYMBOL uniform - GSE65216

In [23]:
# This dataset has /// for synonyms in gene symbols.
# Picking first one for now
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
geneSymbolsWithSlashes = geoData['GSE65216']['platformData']['GENE_SYMBOL'].notna() & \
                         geoData['GSE65216']['platformData']['GENE_SYMBOL'].str.contains('///', na=False)

geoData['GSE65216']['platformData'].loc[geneSymbolsWithSlashes, 'GENE_SYMBOL'] = \
    geoData['GSE65216']['platformData'].loc[geneSymbolsWithSlashes, 'GENE_SYMBOL'].apply(lambda x: x.split(' ///')[0])

print(f"***- GENE_SYMBOL uniform - GSE65216 - Done")

***- GENE_SYMBOL uniform - GSE65216 - Done


### Make GENE_SYMBOL uniform - GSE76250

In [24]:
import numpy as np
import re
geneSymbolsWithSlashes = geoData['GSE76250']['platformData']['GENE_SYMBOL'].str.contains('//')

geoData['GSE76250']['platformData'].loc[geneSymbolsWithSlashes, 'GENE_SYMBOL'] = \
    geoData['GSE76250']['platformData'].loc[geneSymbolsWithSlashes, 'GENE_SYMBOL'].apply(lambda x: re.search("//\\s*([^/]+)\\s*//", x).group(1).strip())

geoData['GSE76250']['platformData'].loc[:, 'GENE_SYMBOL'] = \
    geoData['GSE76250']['platformData'].loc[:, 'GENE_SYMBOL'].replace('---', np.nan)

print(f"***- GENE_SYMBOL uniform - GSE76250 - Done")

***- GENE_SYMBOL uniform - GSE76250 - Done


### Platform data has genes with no symbol -> drop from Platform
### Platform data has genes with no symbol -> drop from DataSet

In [25]:
total = len(geoData.keys())
counter = 1
print(f"***- Removing all Gene Symbols with NaN - Start")

for k in geoData.keys():
    columnsWithNa = geoData[k]['platformData'][geoData[k]['platformData']['GENE_SYMBOL'].isna()]['ID']
    geoData[k]['platformData'].drop(geoData[k]['platformData'][geoData[k]['platformData']['ID'].isin(columnsWithNa)].index, inplace=True)
    geoData[k]['dataSetData'].drop(geoData[k]['dataSetData'][geoData[k]['dataSetData']['ID'].isin(columnsWithNa)].index, inplace=True)
    print(f"{counter}/{total} - Dataset {k} - Removed Gene Symbols n/a")
    counter+=1

print(f"***- Removing all Gene Symbols with NaN - End")

***- Removing all Gene Symbols with NaN - Start
1/2 - Dataset GSE65216 - Removed Gene Symbols n/a
2/2 - Dataset GSE76250 - Removed Gene Symbols n/a
***- Removing all Gene Symbols with NaN - End


# Create DataFrame combined

In [26]:
tnbcStati = ['nTNBC', 'TNBC']
total = len(geoData.keys())
counter = 1

print(f"***- Writing pivotted data - Start")

for k in geoData.keys():
    # Merge Platform and DataSet
    joinedPlatformAndDataSet = pd.merge(geoData[k]['dataSetData'], geoData[k]['platformData'], how='inner', on=["ID"])
    
    # Get column names
    columnsRename = {}
    
    for n in joinedPlatformAndDataSet.index.tolist():
        columnsRename[n] = joinedPlatformAndDataSet['GENE_SYMBOL'][n]
    
    isFirst = True # to circumvent pandas error 'empty cannot concat'
    for tnbc in tnbcStati:
        for geoCase in geoData[k][tnbc]:
            if isFirst:
                intermediate = joinedPlatformAndDataSet[geoCase].to_frame().T.rename(columns=columnsRename).copy(deep=True)
                intermediate['tnbc'] = tnbc == 'TNBC'
                intermediate['case_id'] = geoCase
                casesAsRows = intermediate
                isFirst = False
            else:
                intermediate = joinedPlatformAndDataSet[geoCase].to_frame().T.rename(columns=columnsRename).copy(deep=True)
                intermediate['tnbc'] = tnbc == 'TNBC'
                intermediate['case_id'] = geoCase
                casesAsRows = pd.concat([casesAsRows, intermediate])

    casesAsRows.to_csv(f'../Data/geoData_{k}.csv', index=False)
    print(f"{counter}/{total} - Dataset {k} - Resulting data written")
    counter+=1
    
print(f"***- Writing pivotted data - End")

***- Writing pivotted data - Start
1/2 - Dataset GSE65216 - Resulting data written
2/2 - Dataset GSE76250 - Resulting data written
***- Writing pivotted data - End
